# 🏀 NBA Betting Dashboard Pro

Dashboard interactif pour le système de betting NBA.

**Fonctionnalités :**
- 📊 Visualisation temps réel de la bankroll
- 🎯 Détection des value bets
- 📈 Comparaison des stratégies de mise
- 🏆 Classement des bookmakers
- 💰 Recommandations de paris

**Profils de risque :**
- 🛡️ Conservateur (1% de mise)
- ⚖️ Modéré (2% de mise)
- 🚀 Agressif (5% de mise)

In [ ]:
# Cell 1: Setup et imports
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from ipywidgets import widgets, interact, Layout
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

from src.betting import BettingSystem, OddsClient
from src.reporting.weekly_betting_report import WeeklyBettingReport

print('✅ Modules chargés avec succès!')

## 🎮 Configuration

In [ ]:
# Cell 2: Widgets de configuration

# Sélection du profil de risque
risk_profile_widget = widgets.Dropdown(
    options=[
        ('🛡️ Conservateur (1%)', 'conservative'),
        ('⚖️ Modéré (2%)', 'moderate'),
        ('🚀 Agressif (5%)', 'aggressive')
    ],
    value='moderate',
    description='Profil:',
    style={'description_width': 'initial'}
)

# Bankroll initiale
bankroll_widget = widgets.FloatSlider(
    value=100.0,
    min=50.0,
    max=1000.0,
    step=10.0,
    description='Bankroll (€):',
    style={'description_width': 'initial'}
)

# Stratégie de mise
strategy_widget = widgets.Dropdown(
    options=[
        ('Flat Betting', 'flat'),
        ('Kelly Criterion', 'kelly'),
        ('Confidence-Weighted', 'confidence'),
        ('Value Betting', 'value'),
        ('Martingale (⚠️ Risqué)', 'martingale')
    ],
    value='kelly',
    description='Stratégie:',
    style={'description_width': 'initial'}
)

# Seuil de confiance
confidence_widget = widgets.FloatSlider(
    value=0.65,
    min=0.55,
    max=0.85,
    step=0.05,
    description='Confiance min:',
    style={'description_width': 'initial'}
)

# Bouton d'initialisation
init_button = widgets.Button(
    description='🚀 Initialiser le système',
    button_style='success',
    layout=Layout(width='300px')
)

display(widgets.VBox([
    widgets.HBox([risk_profile_widget, bankroll_widget]),
    widgets.HBox([strategy_widget, confidence_widget]),
    init_button
]))

betting_system = None

def on_init_button_clicked(b):
    global betting_system
    
    print(f"\n🔄 Initialisation du système...")
    print(f"   Profil: {risk_profile_widget.value}")
    print(f"   Bankroll: {bankroll_widget.value}€")
    print(f"   Stratégie: {strategy_widget.value}")
    
    betting_system = BettingSystem(
        initial_bankroll=bankroll_widget.value,
        risk_profile=risk_profile_widget.value
    )
    
    print(f"\n✅ Système initialisé avec succès!")
    print(f"   Balance actuelle: {betting_system.bankroll.current_amount:.2f}€")
    print(f"   ROI: {betting_system.bankroll.roi:+.1f}%")

init_button.on_click(on_init_button_clicked)

## 💰 Vue d'ensemble de la Bankroll

In [ ]:
# Cell 3: Visualisation bankroll

@interact
def plot_bankroll(timeframe=['1W', '1M', '3M', 'ALL']):
    if betting_system is None:
        print("⚠️ Veuillez d'abord initialiser le système (Cell 2)")
        return
    
    transactions = betting_system.bankroll.transactions
    
    if not transactions:
        print("ℹ️ Pas encore de transactions. Simulons quelques paris...")
        # Simulation pour la démo
        import random
        balance = betting_system.bankroll.initial_amount
        dates = [datetime.now() - timedelta(days=x) for x in range(30, 0, -1)]
        
        for date in dates:
            stake = 2.0
            result = 'win' if random.random() > 0.45 else 'loss'  # 55% win rate
            odds = random.uniform(1.70, 2.10)
            profit = stake * (odds - 1) if result == 'win' else -stake
            balance += profit
            transactions.append({
                'date': date.isoformat(),
                'stake': stake,
                'result': result,
                'profit': profit,
                'balance': balance
            })
    
    df = pd.DataFrame(transactions)
    df['date'] = pd.to_datetime(df['date'])
    
    # Crée le graphique
    fig = go.Figure()
    
    # Ligne de bankroll
    fig.add_trace(go.Scatter(
        x=df['date'],
        y=df['balance'],
        mode='lines',
        name='Bankroll',
        line=dict(color='#17408B', width=3)
    ))
    
    # Ligne initiale
    fig.add_hline(y=betting_system.bankroll.initial_amount, 
                  line_dash="dash", 
                  line_color="gray",
                  annotation_text="Initial")
    
    # Points des paris gagnants/perdants
    wins = df[df['result'] == 'win']
    losses = df[df['result'] == 'loss']
    
    fig.add_trace(go.Scatter(
        x=wins['date'],
        y=wins['balance'],
        mode='markers',
        name='Gains ✅',
        marker=dict(color='green', size=8)
    ))
    
    fig.add_trace(go.Scatter(
        x=losses['date'],
        y=losses['balance'],
        mode='markers',
        name='Pertes ❌',
        marker=dict(color='red', size=8)
    ))
    
    fig.update_layout(
        title='Évolution de la Bankroll',
        xaxis_title='Date',
        yaxis_title='Balance (€)',
        hovermode='x unified',
        template='plotly_white',
        height=500
    )
    
    # Métriques
    summary = betting_system.bankroll.get_summary()
    print(f"\n📊 Métriques actuelles:")
    print(f"   Balance: {summary['current']:.2f}€ ({summary['profit_loss']:+.2f}€)")
    print(f"   ROI: {summary['roi_pct']:+.1f}%")
    print(f"   Paris total: {summary['total_bets']}")
    print(f"   Max Drawdown: {summary['max_drawdown_pct']:.1f}%")
    
    fig.show()

## 🎯 Value Bets - Opportunités du jour

In [ ]:
# Cell 4: Tableau des value bets

@interact
def show_value_bets(min_edge=widgets.FloatSlider(
    value=0.05,
    min=0.01,
    max=0.20,
    step=0.01,
    description='Edge min:',
    style={'description_width': 'initial'}
)):
    if betting_system is None:
        print("⚠️ Veuillez d'abord initialiser le système (Cell 2)")
        return
    
    print(f"\n🔍 Recherche des value bets (edge > {min_edge:.0%})...\n")
    
    value_bets = list(betting_system.find_value_bets(min_edge=min_edge))
    
    if not value_bets:
        print("❌ Aucun value bet trouvé avec ces critères")
        return
    
    # Prépare le tableau
    data = []
    for pred, edge, odds in value_bets:
        stake = betting_system.calculate_stake(
            pred, 
            strategy=strategy_widget.value,
            bookmaker_odds=odds
        )
        
        data.append({
            'Match': f"{pred['home_team']} vs {pred['away_team']}",
            'Prédiction': pred['prediction'],
            'Confiance': f"{pred.get('confidence', 0):.0%}",
            'Edge': f"{edge:.1%}",
            'Cote': f"{odds:.2f}",
            'Mise': f"{stake:.2f}€",
            'EV': f"{stake * edge:.2f}€"
        })
    
    df = pd.DataFrame(data)
    
    # Style du tableau
    def color_edge(val):
        edge = float(val.replace('%', ''))
        if edge >= 10:
            return 'background-color: lightgreen; font-weight: bold'
        elif edge >= 5:
            return 'background-color: lightyellow'
        return ''
    
    styled_df = df.style.applymap(color_edge, subset=['Edge'])
    display(styled_df)
    
    print(f"\n✅ {len(value_bets)} opportunités détectées")
    print(f"💡 Misez uniquement si vous êtes confiant dans l'analyse")

## 📈 Comparaison des Stratégies

In [ ]:
# Cell 5: Comparaison des stratégies

def compare_strategies():
    if betting_system is None:
        print("⚠️ Veuillez d'abord initialiser le système (Cell 2)")
        return
    
    strategies = ['flat', 'kelly', 'confidence', 'value', 'martingale']
    
    # Prédiction exemple pour comparaison
    sample_pred = {
        'home_team': 'Sample',
        'away_team': 'Team',
        'prediction': 'Home Win',
        'proba_home_win': 0.70,
        'confidence': 0.70
    }
    bookmaker_odds = 1.85
    
    data = []
    for strategy in strategies:
        try:
            stake = betting_system.calculate_stake(
                sample_pred,
                strategy=strategy,
                bookmaker_odds=bookmaker_odds
            )
            
            descriptions = {
                'flat': 'Mise fixe stable',
                'kelly': 'Optimale mathématiquement',
                'confidence': 'Basée sur confiance ML',
                'value': 'Uniquement si edge > 5%',
                'martingale': '⚠️ Augmente après perte'
            }
            
            data.append({
                'Stratégie': strategy.upper(),
                'Mise (€)': round(stake, 2),
                '% Bankroll': round(stake / betting_system.bankroll.current_amount * 100, 1),
                Description': descriptions[strategy]
            })
        except Exception as e:
            print(f"Erreur {strategy}: {e}")
    
    df = pd.DataFrame(data)
    
    # Graphique
    fig = px.bar(
        df,
        x='Stratégie',
        y='Mise (€)',
        title='Comparaison des Stratégies de Mise',
        color='Mise (€)',
        color_continuous_scale='RdYlGn',
        text='Mise (€)'
    )
    
    fig.update_traces(texttemplate='%{text:.2f}€', textposition='outside')
    fig.update_layout(height=400, showlegend=False)
    
    display(df)
    fig.show()

compare_strategies()

## 🏆 Performance par Niveau de Confiance

In [ ]:
# Cell 6: Analyse des seuils de confiance

def analyze_confidence_thresholds():
    if betting_system is None:
        print("⚠️ Veuillez d'abord initialiser le système (Cell 2)")
        return
    
    print("\n📊 Analyse des seuils de confiance...\n")
    
    thresholds = [0.60, 0.65, 0.70, 0.75, 0.80]
    results = betting_system.test_confidence_thresholds(thresholds)
    
    if 'error' in results:
        print(f"❌ Erreur: {results['error']}")
        return
    
    # Prépare les données pour le graphique
    data = []
    for key, value in results.items():
        if 'accuracy' in value:
            data.append({
                'Seuil': f"{value['threshold']:.0%}",
                'Accuracy': value['accuracy'] * 100,
                'Nb Paris': value['n_predictions'],
                'ROI/Pari': value['roi_per_bet']
            })
    
    if not data:
        print("❌ Pas assez de données historiques")
        return
    
    df = pd.DataFrame(data)
    
    # Graphique combiné
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Accuracy par Seuil', 'Volume de Paris'),
        specs=[[{'secondary_y': False}, {'secondary_y': False}]]
    )
    
    # Accuracy
    fig.add_trace(
        go.Bar(x=df['Seuil'], y=df['Accuracy'], name='Accuracy %',
               marker_color='#17408B'),
        row=1, col=1
    )
    
    # Volume
    fig.add_trace(
        go.Bar(x=df['Seuil'], y=df['Nb Paris'], name='Nb Paris',
               marker_color='#C9082A'),
        row=1, col=2
    )
    
    fig.update_layout(
        title_text="Performance par Seuil de Confiance",
        height=400,
        showlegend=False
    )
    
    display(df)
    fig.show()
    
    # Recommandation
    best = max(data, key=lambda x: x['Accuracy'] if x['Nb Paris'] >= 20 else 0)
    print(f"\n🎯 Recommandation: Utiliser le seuil {best['Seuil']} ({best['Accuracy']:.1f}% accuracy)")

analyze_confidence_thresholds()

## 📧 Générer et Envoyer le Rapport Hebdomadaire

In [ ]:
# Cell 7: Génération du rapport

generate_button = widgets.Button(
    description='📊 Générer le rapport',
    button_style='primary',
    layout=Layout(width='200px')
)

send_email_button = widgets.Button(
    description='📧 Envoyer par email',
    button_style='warning',
    layout=Layout(width='200px')
)

display(widgets.HBox([generate_button, send_email_button]))

def on_generate_clicked(b):
    if betting_system is None:
        print("⚠️ Veuillez d'abord initialiser le système (Cell 2)")
        return
    
    print("\n🔄 Génération du rapport hebdomadaire...\n")
    
    report_gen = WeeklyBettingReport(betting_system)
    files = report_gen.generate_and_save()
    
    print(f"✅ Rapport généré:")
    print(f"   JSON: {files['json']}")
    print(f"   CSV: {files['csv']}")
    print(f"   HTML: {files['html']}")
    
    # Affiche résumé
    report = files['report']
    print(f"\n📊 Résumé:")
    print(f"   Bankroll: {report['bankroll_summary']['current']:.2f}€")
    print(f"   ROI: {report['bankroll_summary']['roi_pct']:+.1f}%")
    print(f"   Value bets: {len(report['value_bets'])}")

def on_send_email_clicked(b):
    if betting_system is None:
        print("⚠️ Veuillez d'abord initialiser le système (Cell 2)")
        return
    
    print("\n📧 Envoi du rapport par email...\n")
    
    try:
        report_gen = WeeklyBettingReport(betting_system)
        report_gen.send_email_report()
        print("✅ Rapport envoyé avec succès!")
    except Exception as e:
        print(f"❌ Erreur: {e}")

generate_button.on_click(on_generate_clicked)
send_email_button.on_click(on_send_email_clicked)

## 💡 Recommandations et Alertes

In [ ]:
# Cell 8: Recommandations

def show_recommendations():
    if betting_system is None:
        print("⚠️ Veuillez d'abord initialiser le système (Cell 2)")
        return
    
    print("\n" + "="*70)
    print("🎯 RECOMMANDATIONS DU SYSTÈME")
    print("="*70 + "\n")
    
    # Génère recommandations
    report_gen = WeeklyBettingReport(betting_system)
    report = report_gen.generate()
    
    for i, rec in enumerate(report['recommendations'], 1):
        print(f"{i}. {rec}\n")
    
    # Alertes importantes
    summary = betting_system.bankroll.get_summary()
    
    print("\n" + "="*70)
    print("⚠️  ALERTES")
    print("="*70 + "\n")
    
    if summary['stop_loss_triggered']:
        print("🚨 STOP-LOSS ATTEINT! Arrêter les paris immédiatement.\n")
    
    if summary['roi_pct'] > 20:
        print("✅ Excellent ROI! Envisager de retirer des bénéfices.\n")
    elif summary['roi_pct'] < -15:
        print("⚠️ ROI négatif important. Réduire les mises.\n")
    
    if summary['max_drawdown_pct'] > 25:
        print("⚠️ Drawdown important. Vérifier les stratégies.\n")
    
    if not summary['stop_loss_triggered'] and summary['roi_pct'] >= -15 and summary['max_drawdown_pct'] <= 25:
        print("✅ Aucune alerte critique. Performance normale.\n")

show_recommendations()

---

## 📝 Notes

- Ce dashboard utilise le **paper trading** (simulation) par défaut
- Pour passer en mode réel, configurez votre clé API The Odds API
- Les recommandations sont générées automatiquement basées sur les performances
- Exportez régulièrement vos rapports pour analyse historique

**Contact:** isaakdjedje@gmail.com